In [1]:
import pandas as pd
import numpy as np

In [2]:
non_afp = pd.read_csv(r"dataset\ad_AFP11.csv").to_numpy().T[1:].T
afp = pd.read_csv(r"dataset\UNSW_NB15_traintest_backdoor.csv").to_numpy().T[1:].T
print(non_afp.shape,afp.shape)

C:\Users\Ricardo Wu\AppData\Local\Temp\ipykernel_12436\2588833736.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  afp = pd.read_csv(r"C:\Users\Ricardo Wu\Desktop\RPCP\ad_Non-AFP11.csv").to_numpy().T[1:].T


(481, 1741) (9190, 1741)


In [3]:
from RPCP import RPEnsemble_CP
from Base_classify import base
from Aggregate import AggregatedCP

In [4]:
n_exp = 5
exp_model = "RandomForest"
alpha = 0.05
base_acc = []
rpecp_acc, rpecp_coverage, rpecp_length = [], [], [] 
icp_acc, icp_coverage, icp_length = [], [], []
ccp_acc, ccp_coverage, ccp_length = [], [], [] 
bcp_acc, bcp_coverage, bcp_length = [], [], [] 

for exp in range(n_exp):
    print("exp:", exp+1, "start:")
    np.random.seed(2024+exp)
    # shuffle_1 = np.random.permutation(np.arange(len(non_afp)))
    # shuffle_2 = np.random.permutation(np.arange(len(afp)))
    # non_afp = non_afp[shuffle_1]
    # afp = afp[shuffle_2]
    

    X_train = np.vstack((non_afp[exp:200+exp],afp[exp:200+exp])),np.concatenate((np.repeat(0,200),np.repeat(1,200)))
    X_test = np.vstack((non_afp[200+exp:400+exp],afp[200+exp:400+exp])), np.concatenate((np.repeat(0,200),np.repeat(1,200)))

    rpecp = RPEnsemble_CP(rp="Gaussian", d = 300, B1=20, B2=50, classifier=exp_model, alpha=alpha)
    rpecp_res = rpecp.RP_Conformal(X_train, X_test)
    rpecp_acc.append(rpecp_res[0])
    rpecp_coverage.append(rpecp_res[1])
    rpecp_length.append(rpecp_res[2])
    print(rpecp_res)
    print("RPECP completed")

    base_classifier = base(classifier = exp_model, random_state=321)
    base_res = base_classifier.predict(X_train, X_test)
    base_acc.append(base_res)
    print(base_res)
    print("Base completed")

    ICPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="ICP")
    icp_res = ICPredictor.predict(X_train, X_test)
    icp_acc.append(icp_res[0])
    icp_coverage.append(icp_res[1])
    icp_length.append(icp_res[2])
    print("ICP completed")

    CCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="CCP")
    ccp_res = CCPredictor.predict(X_train, X_test)
    ccp_acc.append(ccp_res[0])
    ccp_coverage.append(ccp_res[1])
    ccp_length.append(ccp_res[2])
    print("CCP completed")

    BCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="BCP")
    bcp_res = BCPredictor.predict(X_train, X_test)
    bcp_acc.append(bcp_res[0])
    bcp_coverage.append(bcp_res[1])
    bcp_length.append(bcp_res[2])
    print("BCP completed")

    print("---------------------------------------\n")

exp: 1 start:
(0.7175, 0.975, 1.7974358974358975)
RPECP completed
0.7675
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 2 start:
(0.7275, 0.9575, 1.783289817232376)
RPECP completed
0.755
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 3 start:
(0.7675, 0.96, 1.7369791666666667)
RPECP completed
0.785
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 4 start:
(0.725, 0.975, 1.7769230769230768)
RPECP completed
0.7675
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 5 start:
(0.745, 0.96, 1.7890625)
RPECP completed
0.79
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------



In [5]:
print("base acc:", np.mean(base_acc), np.std(base_acc))
print("-----------------------------------------------------")
print("RPECP acc:", np.mean(rpecp_acc), np.std(rpecp_acc))
print("RPECP coverage:", np.mean(rpecp_coverage), np.std(rpecp_coverage))
print("RPECP length:", np.mean(rpecp_length), np.std(rpecp_length))
print("-----------------------------------------------------")
print("ICP acc:", np.mean(icp_acc), np.std(icp_acc))
print("ICP coverage:", np.mean(icp_coverage), np.std(icp_coverage))
print("ICP length:", np.mean(icp_length), np.std(icp_length))
print("-----------------------------------------------------")
print("CCP acc:", np.mean(ccp_acc), np.std(ccp_acc))
print("CCP coverage:", np.mean(ccp_coverage), np.std(ccp_coverage))
print("CCP length:", np.mean(ccp_length), np.std(ccp_length))
print("-----------------------------------------------------")
print("BCP acc:", np.mean(bcp_acc), np.std(bcp_acc))
print("BCP coverage:", np.mean(bcp_coverage), np.std(bcp_coverage))
print("BCP length:", np.mean(bcp_length), np.std(bcp_length))

base acc: 0.773 0.012786711852544442
-----------------------------------------------------
RPECP acc: 0.7365 0.017930421077041087
RPECP coverage: 0.9655000000000001 0.00781024967590665
RPECP length: 1.7767380916516031 0.02099534066890374
-----------------------------------------------------
ICP acc: 0.033 0.017986105748604948
ICP coverage: 0.9984999999999999 0.0029999999999999805
ICP length: 1.9669433249370276 0.01800063676145381
-----------------------------------------------------
CCP acc: 0.008 0.004
CCP coverage: 0.999 0.001224744871391563
CCP length: 1.9919924812030074 0.004000639078098578
-----------------------------------------------------
BCP acc: 0.022 0.016077935190813526
BCP coverage: 1.0 0.0
BCP length: 1.9779999999999998 0.01607793519081355
